In [93]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
train_data = np.loadtxt(open("./all/train.csv", "rb"), delimiter=",", skiprows=1)
test_data = np.loadtxt(open("./all/test.csv", "rb"), delimiter=",", skiprows=1)

In [13]:
y_train = (train_data[:,0]).reshape((42000, 1))
x_train = (train_data[:,1:]).reshape((42000,28,28))

In [20]:
x_train = x_train / 255.0

In [22]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [23]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
42000/42000 [==============================] - 4s 104us/step - loss: 0.2965 - acc: 0.9147
Epoch 2/5
42000/42000 [==============================] - 2s 47us/step - loss: 0.1318 - acc: 0.9614
Epoch 3/5
42000/42000 [==============================] - 2s 46us/step - loss: 0.0895 - acc: 0.9721
Epoch 4/5
42000/42000 [==============================] - 2s 46us/step - loss: 0.0677 - acc: 0.9794
Epoch 5/5
42000/42000 [==============================] - 2s 47us/step - loss: 0.0523 - acc: 0.9840


In [26]:
x_test = test_data.reshape((28000,28,28))

In [30]:
y_test = model.predict_classes(x_test)

In [32]:
y_test

array([2, 0, 9, ..., 3, 9, 2], dtype=int64)

In [34]:
with open("./nn_submission.csv", "w+") as f:
    f.write("ImageId,Label\n")
    for i, label in enumerate(y_test):
        f.write("%s,%s\n" % (i+1, label))

In [69]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [124]:
x_train_cnn = x_train.reshape((42000, 28,28,1))
nets = []
cnn_model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(32, 5, strides=2, padding='same', activation='relu'),

    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Conv2D(64, 5, strides=2, padding='same', activation='relu'),
    keras.layers.Dropout(0.4),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
cnn_model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
y_train_cnn = keras.utils.to_categorical(y_train, num_classes=10)
cnn_model.fit_generator(datagen.flow(x_train_cnn, y_train_cnn, batch_size=64), epochs = 20)

Epoch 1/20
657/657 [==============================] - 8s 13ms/step - loss: 0.2861 - acc: 0.9143: 1s - 
Epoch 2/20
657/657 [==============================] - 8s 12ms/step - loss: 0.1267 - acc: 0.9636
Epoch 3/20
657/657 [==============================] - 8s 13ms/step - loss: 0.0950 - acc: 0.9728
Epoch 4/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0760 - acc: 0.9783
Epoch 5/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0686 - acc: 0.9805
Epoch 6/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0575 - acc: 0.9835
Epoch 7/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0563 - acc: 0.9837
Epoch 8/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0545 - acc: 0.9841
Epoch 9/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0482 - acc: 0.9867
Epoch 10/20
657/657 [==============================] - 8s 12ms/step - loss: 0.0491 - acc: 0.9865
Epoch 11/20
657/657 [===========

In [121]:
x_test_cnn = x_test.reshape((28000, 28, 28, 1))
y_test_cnn = cnn_model.predict_classes(x_test_cnn)

In [122]:
all(y_test_cnn == y_test)

False

In [123]:
with open("./cnn_submission.csv", "w+") as f:
    f.write("ImageId,Label\n")
    for i, label in enumerate(y_test_cnn):
        f.write("%s,%s\n" % (i+1, label))